In [22]:
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import os 
import sys
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras.models import load_model

sys.path.append("../")
os.chdir("/content/drive/My Drive/hackthon_v3")

from src.data.make_dataset import INDEX_SHEET_NAME,load_data
from src.features.build_features import generate_features
from src.models.LSTM import build_lstm_model, generate_train_val_data
from src.models.metrics import LinearCorrelation, MeanAbsolutePercentageError, TheilU

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
INDEX_SHEET_NAME

['HangSeng Index Data',
 'S&P500 Index Data',
 'CSI300 Index Data',
 'DJIA index Data',
 'Nikkei 225 index Data',
 'Nifty 50 index Data']

In [ ]:
raw = load_data(sheet_name=INDEX_SHEET_NAME[-1])
raw.head()

,Date,Ntime,Open Price,High Price,Low price,Closing Price,Volume,MACD,CCI,ATR,...,EMA20,MA5,MA10,MTM6,MTM12,ROC,SMI,WVAD,US dollar Index,Interbank Offered Rate
0,20080701,733590,4039.75,4075.40,3878.20,3896.75,164469220,-201.56,-180.03,197.2,...,4385.058946,4128.49,4253.415,-369.65,-620.35,-13.733369,-0.149627,-828023961.0,72.34,8.706
1,20080702,733591,3895.30,4107.15,3848.25,4093.35,199920144,-199.63,-131.91,258.9,...,4357.277142,4096.63,4204.510,-97.75,-479.15,-10.478950,-0.091225,-561033032.0,71.99,7.730
2,20080703,733592,4094.60,4097.35,3874.85,3925.75,154573765,-209.21,-129.55,222.5,...,4316.179319,4018.61,4146.660,-326.90,-727.25,-15.629701,-0.097020,-726253640.3,72.73,6.400
3,20080704,733593,3926.65,4033.50,3896.40,4016.00,152045352,-207.14,-111.08,137.1,...,4287.590812,3994.48,4113.505,-299.85,-566.40,-12.360335,-0.058691,-548766472.1,72.71,6.210
4,20080707,733596,4002.00,4114.50,4002.00,4030.00,125737237,-202.03,-80.86,112.5,...,4263.058354,3992.37,4089.865,-106.65,-474.25,-10.528945,-0.038439,-486195115.1,72.71,9.000


In [6]:
import os
import sys
import pandas as pd
sys.path.append("../")
RAW_DATA_DIR = "/content/drive/My Drive/hackthon_v3/data/raw"
raw = pd.read_csv(RAW_DATA_DIR + '/BTCUSDT-1s-2022-10-19.csv')
raw.columns = ['Ntime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
raw.drop(labels=["Close time", "Ignore"], axis=1,inplace = True)
mid = raw["Close"]
raw.drop(labels=["Close"], axis=1,inplace = True)
raw.insert(1, "Close", mid)
raw

,Ntime,Close,Open,High,Low,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,1666137601000,19325.17,19323.28,19325.18,19323.28,4.55130,87949.160108,65,4.44719,85937.354853
1,1666137602000,19324.79,19325.18,19325.25,19323.34,0.75719,14632.070033,39,0.20426,3947.369406
2,1666137603000,19321.07,19324.79,19324.79,19320.71,8.65380,167211.173746,163,0.34651,6695.949337
3,1666137604000,19322.81,19321.92,19323.46,19321.21,3.11323,60156.846152,77,2.67632,51714.780183
4,1666137605000,19321.85,19322.07,19323.47,19321.82,1.04121,20119.486277,46,1.00001,19323.422006
...,...,...,...,...,...,...,...,...,...,...
86394,1666223995000,19123.55,19123.57,19123.61,19122.90,0.28020,5358.364997,19,0.11604,2219.086114
86395,1666223996000,19123.70,19123.57,19123.70,19123.57,0.21391,4090.743720,14,0.15967,3053.475239
86396,1666223997000,19122.70,19123.67,19123.67,19122.70,0.09825,1878.818872,20,0.00160,30.597152
86397,1666223998000,19123.55,19122.70,19123.55,19122.41,0.72993,13958.120236,76,0.25358,4849.110043


## The following sections are training process of each model.

### WSAE-LSTM

In [8]:
result_dict = dict()
EPOCHS = 10 #0
past_history = 4

INDEX_SHEET_NAME=['BIT_2022_10_19']

for index in INDEX_SHEET_NAME:
    print(f"Start {index} part!")
    result_dict[index] = dict()

    data_dir = f'/content/drive/My Drive/hackthon_v3/data/processed/wsae/{index}'
    if not os.path.exists(data_dir):
        generate_features(raw, index)
    
    train_lst = os.listdir(data_dir)
    train_lst.sort()
    for name in train_lst:
        x_train = np.load(data_dir + f'/{name}/X_train.npy')
        y_train = np.load(data_dir + f'/{name}/Y_train.npy')
        x_val = np.load(data_dir + f'/{name}/X_val.npy')
        y_val = np.load(data_dir + f'/{name}/Y_val.npy')
        x_test = np.load(data_dir + f'/{name}/X_test.npy')
        y_test = np.load(data_dir + f'/{name}/Y_test.npy')

        train_data, val_data, test_data = generate_train_val_data(
            x_train, y_train, x_val, y_val, x_test, y_test,
            past_history=4, batch_size=60
        )


        model_save_dir = f'/content/drive/My Drive/hackthon_v3/models/{index}/{name}'
        if not os.path.exists(model_save_dir):
            os.makedirs(model_save_dir)
        if not os.path.exists(model_save_dir+'/wase-lstm_config.json') \
            or not os.path.exists(model_save_dir+'/wase-lstm_weights.h5'):
            print("No existing model, start to train!")
            lstm = build_lstm_model(inputs_shape=[4, 10],
                                    layers=5,
                                    units=[64, 64, 64, 64, 64],
                                    learning_rate=0.05)
            lstm.fit(train_data,
                     epochs=EPOCHS,
                     steps_per_epoch=(y_train.shape[0] // 60),
                     validation_data=val_data,
                     validation_steps=1,
                     verbose=0)
            json_config = lstm.to_json()

            with open(model_save_dir+'/wase-lstm_config.json', 'w') as json_file:
                json_file.write(json_config)
            # Save weights to disk
            lstm.save_weights(model_save_dir+'/wase-lstm_weights.h5')
            print("Model Saved!")
        else:
            with open(model_save_dir+'/wase-lstm_config.json') as json_file:
                json_config = json_file.read()
            lstm = tf.keras.models.model_from_json(json_config)
            lstm.compile(loss='mse',
                              optimizer='Adam',
                              metrics=[
                                  tf.keras.metrics.MeanAbsolutePercentageError(),
                                  MeanAbsolutePercentageError(),
                                  LinearCorrelation(),
                                  TheilU()],
                              lr=0.05
                              )
            lstm.load_weights(model_save_dir+'/wase-lstm_weights.h5')

            print("Model Loaded!")

        result_dict[index][name] = lstm.evaluate(test_data, steps=1)
        tf.keras.backend.clear_session()

        print(f">>>>{index} {name} done!<<<<")
    try:
        with open(f'/content/drive/My Drive/hackthon_v3/notebooks/WSAE-LSTM/{index}_train_result.pickle', 'wb') as handle:
            pickle.dump(result_dict[index], handle, protocol=pickle.HIGHEST_PROTOCOL)
    except:
        print('fail to save!')

with open(f'/content/drive/My Drive/hackthon_v3/notebooks/WSAE-LSTM/train_result.pickle', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Start BIT_2022_10_19 part!
No existing model, start to train!


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2086: UserWarning: Metric MeanAbsolutePercentageError implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2086: UserWarning: Metric LinearCorrelation implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2086: UserWarning: Metric TheilU implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


Model Saved!
1/1 [==============================] - 0s 16ms/step - loss: 0.0012 - mean_absolute_percentage_error: 4.2350 - mape: 0.0424 - r: -0.0136 - theil_u: 0.0256
>>>>BIT_2022_10_19 166613787 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 17ms/step - loss: 0.0075 - mean_absolute_percentage_error: 8.9863 - mape: 0.0899 - r: -0.2595 - theil_u: 0.0605
>>>>BIT_2022_10_19 166613790 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 12ms/step - loss: 0.0406 - mean_absolute_percentage_error: 16.2935 - mape: 0.1629 - r: 0.5924 - theil_u: 0.1270
>>>>BIT_2022_10_19 166613793 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 16ms/step - loss: 0.6101 - mean_absolute_percentage_error: 54.4619 - mape: 0.5446 - r: -0.4336 - theil_u: 0.3839
>>>>BIT_2022_10_19 166613796 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [=================

In [58]:
index = "BIT_2022_10_19"
data_dir = f'/content/drive/My Drive/hackthon_v3/data/processed/wsae/{index}'
train_lst = os.listdir(data_dir)
log_x_test = []
log_y_ground_truth = []
log_y_predict = []
for name in train_lst:
    x_train = np.load(data_dir + f'/{name}/X_train.npy')
    y_train = np.load(data_dir + f'/{name}/Y_train.npy')
    x_val = np.load(data_dir + f'/{name}/X_val.npy')
    y_val = np.load(data_dir + f'/{name}/Y_val.npy')
    x_test = np.load(data_dir + f'/{name}/X_test.npy')
    y_test = np.load(data_dir + f'/{name}/Y_test.npy')
    
    train_data, val_data, test_data = generate_train_val_data( #train_data = 80 with 4 slices -> 6000
        x_train, y_train, x_val, y_val, x_test, y_test,
        past_history=4, batch_size=60
    )

    result = lstm.predict(test_data)
    log_x_test.append(test_data)
    log_y_ground_truth.append(y_test.reshape(-1,1))
    log_y_predict.append(result)
    print("==================" + name + "================")
    # print("This is the ground truth")
    # print(y_test.reshape(-1,1))
    # print("This is the prediction")
    print(result)

串流輸出內容已截斷至最後 5000 行。
 [0.36659315]
 [0.43304008]
 [0.48270163]
 [0.48283616]
 [0.48780307]
 [0.5032853 ]
 [0.504523  ]
 [0.40802917]]
1/1 [==============================] - 0s 16ms/step
==================166614228================
[[0.50128496]
 [0.49930385]
 [0.7802597 ]
 [0.49818775]
 [0.6244792 ]
 [0.65903723]
 [0.5748395 ]
 [0.4661061 ]
 [0.45271334]
 [0.4353315 ]
 [0.19834583]
 [0.29574332]
 [0.45377865]
 [0.453053  ]
 [0.45239416]
 [0.4435584 ]
 [0.45443127]
 [0.46369722]
 [0.25331235]
 [0.55908644]
 [0.48188284]
 [0.48187903]
 [0.48204306]
 [0.4797621 ]
 [0.44396266]
 [0.44459262]]
1/1 [==============================] - 0s 18ms/step
==================166614231================
[[0.5078354 ]
 [0.50785244]
 [0.50799364]
 [0.5085255 ]
 [0.50575674]
 [0.5086674 ]
 [0.5018137 ]
 [0.5015    ]
 [0.504393  ]
 [0.50969166]
 [0.50567985]
 [0.50916016]
 [0.4750907 ]
 [0.4752873 ]
 [0.4937894 ]
 [0.5015381 ]
 [0.6284233 ]
 [0.17863145]
 [0.4647604 ]
 [0.47961214]
 [0.51046884]
 [0.510449  ]
 

In [ ]:
log_new_y_predict = []
for y_predict in log_y_predict:
  temp = [[0],[0], [0], [0]]
  temp = np.array(temp)
  temp = np.concatenate([temp, y_predict], axis = 0)
  log_new_y_predict.append(temp)
  print(len(temp))

log_new_y_predict = np.array(log_new_y_predict[:-1])

In [50]:
log_y_ground_truth = np.array(log_y_ground_truth[:-1])

In [53]:
ny_g = log_y_ground_truth.reshape(-1, 30)
ny_p = log_new_y_predict.reshape(-1, 30)


In [ ]:
df_y_ground = pd.DataFrame(ny_g)
df_y_predict = pd.DataFrame(ny_p)


In [57]:
df_y_ground.to_csv('y_ground.csv',index = False)
df_y_predict.to_csv('y_predict.csv',index = False)

In [26]:
df = [{}]

<BatchDataset element_spec=(TensorSpec(shape=(None, 4, 10), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [10]:
# origin x_test
x_test_origin = x_test.copy()
y_test_origin = y_test.copy()

print(x_test.size)
print(y_test.size)
print(x_test_origin)
print(y_test_origin)

100
10
[[0.61602193 0.7380974  0.70713323 0.51482683 0.76128685 0.22782724
  0.31249028 0.51714176 0.4413728  0.21683165]
 [0.6160857  0.7380911  0.7070847  0.51474255 0.7612997  0.22780392
  0.31246006 0.5171293  0.44138002 0.21687543]
 [0.616021   0.7381633  0.70703864 0.51456136 0.7614584  0.22769184
  0.31233284 0.5175568  0.44114518 0.21660769]
 [0.61671835 0.7380322  0.7071025  0.5142188  0.76147723 0.22800072
  0.31254637 0.51606447 0.44148037 0.21720874]
 [0.6164338  0.7381836  0.7068079  0.5138276  0.76164234 0.22767867
  0.31228548 0.5174161  0.44090304 0.2168384 ]
 [0.61640924 0.73820496 0.7068886  0.5139235  0.7616156  0.22771269
  0.3123083  0.5173625  0.4409887  0.21683069]
 [0.6164456  0.73816514 0.706868   0.51394695 0.7616003  0.22772638
  0.3123392  0.5172237  0.44100845 0.21689329]
 [0.616452   0.7382145  0.7068127  0.5137524  0.7616883  0.22765312
  0.3122269  0.5175003  0.44089904 0.21677391]
 [0.6164287  0.738216   0.7068279  0.51378477 0.7616864  0.22766395
  0.3

In [ ]:
import csv

# open the file in the write mode
with open('path/to/csv_file', 'w') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    writer.writerow(row)

### WLSTM

In [ ]:
result_dict=dict()
EPOCHS = 500 #0
past_history = 4

for index in INDEX_SHEET_NAME:
    print(f"Start {index} part!")
    result_dict[index] = dict()

    data_dir = f'../data/processed/wavelet/{index}'
    if not os.path.exists(data_dir):
        raw = load_data(sheet_name=index)
        generate_features(raw, index)
    
    train_lst = os.listdir(data_dir)
    train_lst.sort()
    for name in train_lst:
        x_train = np.load(data_dir + f'/{name}/X_train.npy')
        y_train = np.load(data_dir + f'/{name}/Y_train.npy')
        x_val = np.load(data_dir + f'/{name}/X_val.npy')
        y_val = np.load(data_dir + f'/{name}/Y_val.npy')
        x_test = np.load(data_dir + f'/{name}/X_test.npy')
        y_test = np.load(data_dir + f'/{name}/Y_test.npy')

        train_data, val_data, test_data = generate_train_val_data(
            x_train, y_train, x_val, y_val, x_test, y_test,
            past_history=4, batch_size=60
        )


        model_save_dir = f'../models/{index}/{name}'
        config_filename = '/wlstm_config.json'
        weight_filename = '/wlstm_weights.h5'

        if not os.path.exists(model_save_dir):
            os.makedirs(model_save_dir)
        if not os.path.exists(model_save_dir+config_filename) \
            or not os.path.exists(model_save_dir+weight_filename):
            print("No existing model, start to train!")
            lstm = build_lstm_model(inputs_shape=[4, 19],
                                    layers=5,
                                    units=[64, 64, 64, 64, 64],
                                    learning_rate=0.05)
            lstm.fit(train_data,
                     epochs=EPOCHS,
                     steps_per_epoch=(y_train.shape[0] // 60),
                     validation_data=val_data,
                     validation_steps=1,
                     verbose=0)
            json_config = lstm.to_json()

            with open(model_save_dir+config_filename, 'w') as json_file:
                json_file.write(json_config)
            # Save weights to disk
            lstm.save_weights(model_save_dir+weight_filename)
            print("Model Saved!")
        else:
            with open(model_save_dir+config_filename) as json_file:
                json_config = json_file.read()
            lstm = tf.keras.models.model_from_json(json_config)
            lstm.compile(loss='mse',
                              optimizer='Adam',
                              metrics=[
                                  tf.keras.metrics.MeanAbsolutePercentageError(),
                                  MeanAbsolutePercentageError(),
                                  LinearCorrelation(),
                                  TheilU()],
                              lr=0.05
                              )
            lstm.load_weights(model_save_dir+weight_filename)

            print("Model Loaded!")

        result_dict[index][name] = lstm.evaluate(test_data, steps=1)
        tf.keras.backend.clear_session()

        print(f">>>>{index} {name} done!<<<<")
    try:
        with open(f'./wlstm/{index}_train_result.pickle', 'wb') as handle:
            pickle.dump(result_dict[index], handle, protocol=pickle.HIGHEST_PROTOCOL)
    except:
        print('fail to save!')

with open(f'./wlstm/train_result.pickle', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Start DJIA index Data part!
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 11ms/step - loss: 0.0013 - mean_absolute_percentage_error: 3.5034 - mape: 0.0350 - r: 0.8963 - theil_u: 0.0201
>>>>DJIA index Data 201010 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 11ms/step - loss: 0.0071 - mean_absolute_percentage_error: 6.6325 - mape: 0.0663 - r: 0.8050 - theil_u: 0.0370
>>>>DJIA index Data 201101 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 11ms/step - loss: 0.0130 - mean_absolute_percentage_error: 9.2717 - mape: 0.0927 - r: 0.6966 - theil_u: 0.0518
>>>>DJIA index Data 201104 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 10ms/step - loss: 0.0032 - mean_absolute_percentage_error: 5.5508 - mape: 0.0555 - r: 0.9062 - theil_u: 0.0335
>>>>DJIA index Data 201107 done!<<<<
No existing model, s

### SAE-LSTM

In [ ]:
result_dict = dict()
EPOCHS = 500 #0
past_history = 4

for index in INDEX_SHEET_NAME:
    print(f"Start {index} part!")
    result_dict[index] = dict()

    data_dir = f'../data/processed/sae/{index}'
    if not os.path.exists(data_dir):
        raw = load_data(sheet_name=index)
        generate_features(raw, index)
    
    train_lst = os.listdir(data_dir)
    train_lst.sort()
    for name in train_lst:
        x_train = np.load(data_dir + f'/{name}/X_train.npy')
        y_train = np.load(data_dir + f'/{name}/Y_train.npy')
        x_val = np.load(data_dir + f'/{name}/X_val.npy')
        y_val = np.load(data_dir + f'/{name}/Y_val.npy')
        x_test = np.load(data_dir + f'/{name}/X_test.npy')
        y_test = np.load(data_dir + f'/{name}/Y_test.npy')

        train_data, val_data, test_data = generate_train_val_data(
            x_train, y_train, x_val, y_val, x_test, y_test,
            past_history=4, batch_size=60
        )


        model_save_dir = f'../models/{index}/{name}'
        config_filename = '/sae-lstm_config.json'
        weight_filename = '/sae-lstm_weights.h5'

        if not os.path.exists(model_save_dir):
            os.makedirs(model_save_dir)
#         if not os.path.exists(model_save_dir+config_filename) \
#             or not os.path.exists(model_save_dir+weight_filename):
        if True:
            print("No existing model, start to train!")
            lstm = build_lstm_model(inputs_shape=[4, 10],
                                    layers=5,
                                    units=[64, 64, 64, 64, 64],
                                    learning_rate=0.05)
            lstm.fit(train_data,
                     epochs=EPOCHS,
                     steps_per_epoch=(y_train.shape[0] // 60),
                     validation_data=val_data,
                     validation_steps=1,
                     verbose=0)
            json_config = lstm.to_json()

            with open(model_save_dir+config_filename, 'w') as json_file:
                json_file.write(json_config)
            # Save weights to disk
            lstm.save_weights(model_save_dir+weight_filename)
            print("Model Saved!")
        else:
            with open(model_save_dir+config_filename) as json_file:
                json_config = json_file.read()
            lstm = tf.keras.models.model_from_json(json_config)
            lstm.compile(loss='mse',
                              optimizer='Adam',
                              metrics=[
                                  tf.keras.metrics.MeanAbsolutePercentageError(),
                                  MeanAbsolutePercentageError(),
                                  LinearCorrelation(),
                                  TheilU()],
                              lr=0.05
                              )
            lstm.load_weights(model_save_dir+weight_filename)

            print("Model Loaded!")

        result_dict[index][name] = lstm.evaluate(test_data, steps=1)
        tf.keras.backend.clear_session()

        print(f">>>>{index} {name} done!<<<<")
    try:
        with open(f'./sae-lstm/{index}_train_result.pickle', 'wb') as handle:
            pickle.dump(result_dict[index], handle, protocol=pickle.HIGHEST_PROTOCOL)
    except:
        print('fail to save!')

with open(f'./sae-lstm/train_result.pickle', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Start Nifty 50 index Data part!
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 10ms/step - loss: 0.0116 - mean_absolute_percentage_error: 8.5186 - mape: 0.0852 - r: 0.9339 - theil_u: 0.0456
>>>>Nifty 50 index Data 201010 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 11ms/step - loss: 0.0032 - mean_absolute_percentage_error: 5.4680 - mape: 0.0547 - r: 0.8130 - theil_u: 0.0329
>>>>Nifty 50 index Data 201101 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 11ms/step - loss: 0.0113 - mean_absolute_percentage_error: 12.4645 - mape: 0.1246 - r: 0.3328 - theil_u: 0.0628
>>>>Nifty 50 index Data 201104 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 11ms/step - loss: 0.0013 - mean_absolute_percentage_error: 4.5141 - mape: 0.0451 - r: 0.9159 - theil_u: 0.0266
>>>>Nifty 50 index Data 201107 done!<<<<

### LSTM

In [ ]:
result_dict=dict()
EPOCHS = 500 #0
past_history = 4

for index in INDEX_SHEET_NAME:
    print(f"Start {index} part!")
    result_dict[index] = dict()

    data_dir = f'../data/interim/{index}'
    if not os.path.exists(data_dir):
        raw = load_data(sheet_name=index)
        generate_features(raw, index)
    
    train_lst = os.listdir(data_dir)
    train_lst.sort()
    for name in train_lst:
        x_train = np.load(data_dir + f'/{name}/X_train.npy')
        y_train = np.load(data_dir + f'/{name}/Y_train.npy')
        x_val = np.load(data_dir + f'/{name}/X_val.npy')
        y_val = np.load(data_dir + f'/{name}/Y_val.npy')
        x_test = np.load(data_dir + f'/{name}/X_test.npy')
        y_test = np.load(data_dir + f'/{name}/Y_test.npy')

        train_data, val_data, test_data = generate_train_val_data(
            x_train, y_train, x_val, y_val, x_test, y_test,
            past_history=4, batch_size=60
        )


        model_save_dir = f'../models/{index}/{name}'
        config_filename = '/lstm_config.json'
        weight_filename = '/lstm_weights.h5'

        if not os.path.exists(model_save_dir):
            os.makedirs(model_save_dir)
        if not os.path.exists(model_save_dir+config_filename) \
            or not os.path.exists(model_save_dir+weight_filename):
            print("No existing model, start to train!")
            lstm = build_lstm_model(inputs_shape=[4, 19],
                                    layers=5,
                                    units=[64, 64, 64, 64, 64],
                                    learning_rate=0.05)
            lstm.fit(train_data,
                     epochs=EPOCHS,
                     steps_per_epoch=(y_train.shape[0] // 60),
                     validation_data=val_data,
                     validation_steps=1,
                     verbose=0)
            json_config = lstm.to_json()

            with open(model_save_dir+config_filename, 'w') as json_file:
                json_file.write(json_config)
            # Save weights to disk
            lstm.save_weights(model_save_dir+weight_filename)
            print("Model Saved!")
        else:
            with open(model_save_dir+config_filename) as json_file:
                json_config = json_file.read()
            lstm = tf.keras.models.model_from_json(json_config)
            lstm.compile(loss='mse',
                              optimizer='Adam',
                              metrics=[
                                  tf.keras.metrics.MeanAbsolutePercentageError(),
                                  MeanAbsolutePercentageError(),
                                  LinearCorrelation(),
                                  TheilU()],
                              lr=0.05
                              )
            lstm.load_weights(model_save_dir+weight_filename)

            print("Model Loaded!")

        result_dict[index][name] = lstm.evaluate(test_data, steps=1)
        tf.keras.backend.clear_session()

        print(f">>>>{index} {name} done!<<<<")
    try:
        with open(f'./lstm/{index}_train_result.pickle', 'wb') as handle:
            pickle.dump(result_dict[index], handle, protocol=pickle.HIGHEST_PROTOCOL)
    except:
        print('fail to save!')

with open(f'./lstm/train_result.pickle', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Start DJIA index Data part!
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 10ms/step - loss: 2.9012e-04 - mean_absolute_percentage_error: 1.6065 - mape: 0.0161 - r: 0.9727 - theil_u: 0.0095
>>>>DJIA index Data 201010 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 12ms/step - loss: 0.0111 - mean_absolute_percentage_error: 8.5548 - mape: 0.0855 - r: 0.8294 - theil_u: 0.0466
>>>>DJIA index Data 201101 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 11ms/step - loss: 0.0081 - mean_absolute_percentage_error: 7.5488 - mape: 0.0755 - r: 0.9439 - theil_u: 0.0404
>>>>DJIA index Data 201104 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 11ms/step - loss: 8.4571e-04 - mean_absolute_percentage_error: 2.8125 - mape: 0.0281 - r: 0.9823 - theil_u: 0.0173
>>>>DJIA index Data 201107 done!<<<<
No existing 